In [8]:
import jieba.posseg as pseg
import jieba
import operator
import pandas as pd
import numpy as np
import jieba.analyse

In [3]:
#함수 목록들

# 리스트의 원소에서 중복을 제거한다
def list_repeat_removal(input_list):
    res = []
    for i in input_list:
        if i not in res:
            res.append(i)
    return res

def reps_list_maker(keyterms_df):
    reps_list = list_repeat_removal(keyterms_df[keyterms_df['대표어'].isnull() == False]['대표어']) #동의어, 서술어 제외
    reps_exp_list = list_repeat_removal(keyterms_df[keyterms_df['대표어(서술어)'].isnull() == False]['대표어(서술어)']) #대표어, 서술어만

    reps_all_list = reps_list.copy()
    reps_all_list.extend(reps_exp_list)
    
    return reps_list, reps_exp_list, reps_all_list

#동의어 작업을 위한 전처리 과정1 - 모든 동의어에 대한 list를 만든다

#우선 동의어가 있는 대표어들을 추릴 수 있도록 Dataframe을 만든다.
def syns_list_maker(keyterms_df):
    exist_syn = keyterms_df[keyterms_df['동의어'].isnull()== False][['대표어', '동의어']]
    exist_syn_exp = keyterms_df[keyterms_df['동의어(서술어)'].isnull() == False][['대표어(서술어)', '동의어(서술어)']]

    syns_list = list_repeat_removal(exist_syn['동의어'])
    syns_exp_list = list_repeat_removal(exist_syn_exp['동의어(서술어)'])

    syns_all_list = syns_list.copy()
    syns_all_list.extend(syns_exp_list)
    
    #동의어 작업을 위한 전처리 과정2 - 각각의 대표어를 key로, 대표어에 대한 동의어(대표어 자신을 포함) list를 value로 갖는 dictionary를 만든다.
    syn_dict = {}
    reps_syn_list = list_repeat_removal(exist_syn['대표어'])
    reps_syn_exp_list = list_repeat_removal(exist_syn_exp['대표어(서술어)'])
    
    for i in reps_syn_list:
        syns = [i] #자기 자신을 포함
  
        for j in exist_syn[exist_syn['대표어'] == i]['동의어']:
            syns.append(j)

        syn_dict[i] = syns
    
    # 서술어    
    for i in reps_syn_exp_list:
        syns = [i]

        for j in exist_syn_exp[exist_syn_exp['대표어(서술어)'] == i]['동의어(서술어)']:
            syns.append(j)
  
        syn_dict[i] = syns
        
    return syns_list, syns_exp_list, syns_all_list, syn_dict

# 각 문서에 등장한 word를 딱 한 번만 count해서 각 문서에서 등장한 모든 word의 list를 원소로 갖는 list를 반환, 동의어의 경우 대표어만 셈

def word_list_maker(cut_docs, keyterms_df, syns_all_list, syns_list, syns_exp_list):
    word_list_all = []

    for words in cut_docs:
        word_list = []

        for i in words:
            if i in syns_all_list:
                if i in syns_list:
                    rep_temp = keyterms_df[keyterms_df['동의어'] == i]['대표어'].values[0]
                elif i in syns_exp_list:
                    rep_temp = keyterms_df[keyterms_df['동의어(서술어)'] == i]['대표어(서술어)'].values[0]
        
                if rep_temp not in word_list:
                    word_list.append(rep_temp)

            else:
                if i not in word_list:
                    word_list.append(i)

        word_list_all.append(word_list)
  
    return word_list_all

#Word_list_all로부터 등장한 단어의 빈도수를 세서, dictionary로 반환

def k2_freq(word_list_all):
    word_count_dict = {}
    for wl in word_list_all:
        for keyword in wl:
            if keyword in word_count_dict.keys():
                word_count_dict[keyword] = word_count_dict[keyword] + 1
        
            else:
                word_count_dict[keyword] = 1

    res = sorted(word_count_dict.items(), key=lambda x: x[1], reverse=True)
  
    return res

#word_list_all에 이미, 동의어가 등장하는 경우 대표어로만 세기 때문에, 모든 문서에서 등장한 동의어는 대표어로 바뀌어있음

def k1_word_list_maker(k1, word_list_all):
    k1_word_list = [i for i in word_list_all if k1 in i]
    return k1_word_list

#우리로 치면 K1에 대한 연관어, 즉 k2 freq를 세기 위해 k1이 등장한 문서들을 word가 담긴 list로 바꿔준다.
# 동의어와 대표어를 포함해 이들 중에 하나만 등장해도 센다.
#K1이 등장한 문서의 cut_word 값만을 구하고, 이걸 work_list_maker에 넣으면 됨

def make_k1_doc(k1, cut_docs_all, syn_dict):
    k1_doc = []

    if k1 in syn_dict.keys():
        k1_syn_list = syn_dict[k1]

        for words in cut_docs_all:
            if len(set(k1_syn_list)) > len(set(k1_syn_list) -set(words)):
                k1_doc.append(words)

    else:
        for words in cut_docs_all:
            if k1 in cut_docs_all:
                k1_doc.append(words)
  
    return k1_doc

In [4]:
doc_df = pd.read_excel('china_doc_test.xlsx')
doc_list = doc_df['doc']

keyterms_df = pd.read_excel('test2.xlsx', sheet_name='keyterms')

In [5]:
reps_list, reps_exp_list, reps_all_list = reps_list_maker(keyterms_df)
syns_list, syns_exp_list, syns_all_list, syn_dict = syns_list_maker(keyterms_df)

In [7]:
for i in reps_list:
      jieba.add_word(i, tag = 'n')

for i in reps_exp_list:
      jieba.add_word(i, tag = 'v')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\rlawh\AppData\Local\Temp\jieba.cache
Loading model cost 0.571 seconds.
Prefix dict has been built successfully.


In [9]:
#어떤 word가 등장한 문서를 센다, 이때 동의어가 있으면, 대표어와 동의어를 포함해 단 한 번만 센다.
#K2 freq를 세기 위해서 모든 텍스트를 우선 컷해서 담는다.

cut_docs_all = []
for doc in doc_list:
    temp_cut = jieba.lcut(doc, use_paddle=True) #여길 제너레이터가 아니라 리스트로 반드시 바꿔줘야함
    # temp_cut = list(pseg.cut(doc)) #여길 제너레이터가 아니라 리스트로 반드시 바꿔줘야함
    cut_docs_all.append(temp_cut)

In [10]:
word_list_all = word_list_maker(cut_docs_all, keyterms_df, syns_all_list, syns_list, syns_exp_list)b

In [14]:
k1 = input('k1을 입력하세요 ')
k1

k1을 입력하세요 公司


'公司'

In [15]:
k1_word_list = k1_word_list_maker(k1, word_list_all)

In [16]:
res = k2_freq(k1_word_list)
res

[(' ', 1702),
 ('公司', 1702),
 ('，', 1672),
 ('。', 1669),
 ('的', 1640),
 ('...', 1518),
 ('、', 1473),
 ('和', 1141),
 ('年', 1116),
 ('在', 1038),
 ('：', 971),
 ('“', 921),
 ('”', 906),
 ('为', 864),
 ('2021', 831),
 ('了', 819),
 ('等', 777),
 ('及', 747),
 ('（', 712),
 ('与', 711),
 ('）', 711),
 ('对', 698),
 ('有限公司', 661),
 ('是', 649),
 ('日', 610),
 ('企业', 593),
 ('将', 564),
 ('并', 550),
 ('发展', 530),
 (':', 521),
 ('《', 492),
 ('》', 486),
 ('12月', 465),
 ('中', 444),
 ('股份', 430),
 ('通过', 426),
 ('情况', 423),
 ('以', 420),
 ('中国', 414),
 ('于', 414),
 ('其', 398),
 ('本', 397),
 ('公告', 394),
 ('团体', 383),
 ('不', 381),
 ('1', 377),
 ('内容', 376),
 ('项目', 371),
 ('相关', 368),
 ('一', 367),
 ('董事会', 365),
 ('-', 361),
 ('以下', 350),
 ('简称', 346),
 ('上', 345),
 ('有', 343),
 ('建设', 328),
 ('关于', 327),
 ('重大', 327),
 ('该', 323),
 ('新', 317),
 ('保证', 316),
 ('进行', 311),
 ('表示', 300),
 ('；', 299),
 ('2', 295),
 ('根据', 295),
 ('提供', 288),
 ('管理', 288),
 ('技术', 283),
 ('科技', 283),
 ('也', 283),
 ('实现', 281),
 ('